In [16]:
#!pip install web3

In [17]:
import pandas as pd
import requests
import datetime
import time
import json
from web3 import Web3 as web3
import numpy as np
from web3 import Web3

In [ ]:
# Initialize a Web3 instance
infura_project_id = 'YOUR-PROJECT-ID'
infura_url = f'https://mainnet.infura.io/v3/{infura_project_id}'

web3 = Web3(Web3.HTTPProvider(infura_url))

In [4]:
#introduce a caching layer to reduce API calls

from web3.middleware import geth_poa_middleware, simple_cache_middleware
# add the geth_poa_middleware to handle POA chains
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# add the simple_cache_middleware to cache responses
web3.middleware_onion.add(simple_cache_middleware)

In [ ]:
#load top token addresses and block numbers that will be needed later on
top_tokens_addresses = pd.read_csv('../../utils/price-data/top_tokens_addresses.csv') 
block_numbers_df = pd.read_csv('../../utils/block-numbers/block_numbers_2020-01-2023-04.csv')

<h1 style="color: blue; font-size: 36px; font-family: Arial;">  ZkSync ERC20 Escrow</h1>



In [2]:
#contract-address: 0x57891966931Eb4Bb6FB81430E6cE0A03AAbDe063
#created at 14-02-2023 #at block 16626406


In [28]:
import pandas as pd
import requests

# Replace API_KEY with your own Ethplorer API key
API_KEY = "YOUR-API-KEY"

# Replace CONTRACT_ADDRESS with the address of the contract you want to query
CONTRACT_ADDRESS = "0x57891966931Eb4Bb6FB81430E6cE0A03AAbDe063"

# Construct the API endpoint URL
url = f"https://api.ethplorer.io/getAddressInfo/{CONTRACT_ADDRESS}?apiKey={API_KEY}"

# Send the API request and parse the response as JSON
response = requests.get(url)
data = response.json()
# Get the list of token balances held by the contract
token_balances = data.get("tokens", [])

# Create an empty DataFrame
token_addresses_held_erc20 = pd.DataFrame(columns=["Token Address"])

# Append each token address to a new row in the DataFrame
for token_balance in token_balances:
    token_address = token_balance.get("tokenInfo", {}).get("address")
    token_addresses_held_erc20 = token_addresses_held_erc20.append({"Token Address": token_address}, ignore_index=True)

# Print the DataFrame
print(token_addresses_held_erc20)

                                 Token Address
0   0xa49d7499271ae71cd8ab9ac515e6694c755d400c
1   0x471ea49dd8e60e697f4cac262b5fafcc307506e4
2   0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
3   0xbc396689893d065f41bc2c6ecbee5e0085233447
4   0xffffffff2ba8f66d4e51811c5190992176930278
5   0xf655c8567e0f213e6c634cd2a68d992152161dc6
6   0xba100000625a3754423978a60c9317c58a424e3d
7   0x5f98805a4e8be255a32880fdec7f6728c6568ba0
8   0x95b3497bbcccc46a8f45f5cf54b0878b39f8d96c
9   0xc17272c3e15074c55b810bceba02ba0c4481cd79
10  0xf9c53268e9de692ae1b2ea5216e24e1c3ad7cb1e
11  0x63a3ae78711b52fb75a03acf9996f18ab611b877
12  0xdac17f958d2ee523a2206206994597c13d831ec7
13  0xcda4e840411c00a614ad9205caec807c7458a0e3
14  0x5f64ab1544d28732f0a24f4713c2c8ec0da089f0
15  0xa487bf43cf3b10dffc97a9a744cbb7036965d3b9
16  0x4691937a7508860f876c9c0a2a617e7d9e945d4b
17  0xeeaa40b28a2d1b0b08f6f97bb1dd4b75316c6107
18  0xdddddd4301a082e62e84e43f474f044423921918
19  0x111111111117dc0aa78b770fa6a738034120c302
20  0xc63e1f3

In [29]:
# all token_addresses held by the escrow that also appear in the top 300 of CoinGecko
erc20escrow_top_token_addresses = pd.merge(top_tokens_addresses, token_addresses_held_erc20, left_on="token_address", right_on="Token Address")
erc20escrow_top_token_addresses['token_address'] = erc20escrow_top_token_addresses['token_address'].apply(lambda x: web3.to_checksum_address(x))


In [30]:
erc20escrow_top_token_addresses

,coingecko_id,token_address,decimals,Token Address
0,tether,0xdAC17F958D2ee523a2206206994597C13D831ec7,6,0xdac17f958d2ee523a2206206994597c13d831ec7
1,usd-coin,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,6,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
2,shiba-inu,0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE,18,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce
3,binance-usd,0x4Fabb145d64652a948d72533023f6E7A623C7C53,18,0x4fabb145d64652a948d72533023f6e7a623c7c53
4,wrapped-bitcoin,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,8,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
5,rocket-pool,0xD33526068D116cE69F19A9ee46F0bd304F21A51f,18,0xd33526068d116ce69f19a9ee46f0bd304f21a51f
6,woo-network,0x4691937a7508860F876c9c0a2a617E7d9E945D4B,18,0x4691937a7508860f876c9c0a2a617e7d9e945d4b
7,rocket-pool-eth,0xae78736Cd615f374D3085123A210448E74Fc6393,18,0xae78736cd615f374d3085123a210448e74fc6393
8,1inch,0x111111111117dC0aa78b770fA6A738034120C302,18,0x111111111117dc0aa78b770fa6a738034120c302
9,floki,0xcf0C122c6b73ff809C693DB761e7BaeBe62b6a2E,9,0xcf0c122c6b73ff809c693db761e7baebe62b6a2e


In [31]:
#date of the contract deployment
start_date = '2023-02-14'
# Get the index position of the start date in the DataFrame
start_index = block_numbers_df[block_numbers_df['Date'] == start_date].index[0]
# Slice the DataFrame from the start index onwards
block_numbers_slice = block_numbers_df.iloc[start_index:]

#define function to retrieve account balance
def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting balance for {token_address}: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token addresses DataFrame
token_addresses_df = erc20escrow_top_token_addresses

token_holding_address = "0x57891966931Eb4Bb6FB81430E6cE0A03AAbDe063"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for each token address
contracts = {}
for _, row in token_addresses_df.iterrows():
    token_address = row['Token Address']
    token_address_checksum = web3.to_checksum_address(token_address)
    contract = web3.eth.contract(address=token_address_checksum, abi=abi)
    contracts[token_address] = contract

# Retrieve the token balances for each token address and block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_slice.iterrows():
    print(f"Retrieving token balances for {row['Date']} - Block Number:{row['Block Number']} ")
    for _, row2 in token_addresses_df.iterrows(): 
        token_decimals = row2['decimals']
        token_address = row2['Token Address']
        block_number = row['Block Number']
        contract = contracts[token_address]
        token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
        #print(f"On day {row['Date']} token address {token_address} had a balance of {token_balance} ")
        #print(f"Token balance for address {token_address} at Block-Nr. {row['Block Number']} is: {token_balance}")
        time.sleep(0.5)
        balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': row2['coingecko_id'],
            'token_decimals': row2['decimals'],
            'balance': token_balance / 10**token_decimals,
            'has_error': has_error
        })

erc20_escrow_token_balances_per_day = pd.DataFrame(balances)


Retrieving token balances for 2023-02-14 - Block Number:16630422 
Retrieving token balances for 2023-02-15 - Block Number:16637574 
Retrieving token balances for 2023-02-16 - Block Number:16644732 
Retrieving token balances for 2023-02-17 - Block Number:16651829 
Retrieving token balances for 2023-02-18 - Block Number:16658938 
Retrieving token balances for 2023-02-19 - Block Number:16666055 
Retrieving token balances for 2023-02-20 - Block Number:16673184 
Retrieving token balances for 2023-02-21 - Block Number:16680299 
Retrieving token balances for 2023-02-22 - Block Number:16687395 
Retrieving token balances for 2023-02-23 - Block Number:16694513 
Retrieving token balances for 2023-02-24 - Block Number:16701598 
Retrieving token balances for 2023-02-25 - Block Number:16708710 
Retrieving token balances for 2023-02-26 - Block Number:16715829 
Retrieving token balances for 2023-02-27 - Block Number:16722962 
Retrieving token balances for 2023-02-28 - Block Number:16730071 
Retrieving

In [35]:
erc20_escrow_token_balances_per_day.to_csv('Erc20Escrow/erc20_escrow_token_balances_per_day.csv', index=False)


<h1 style="color: blue; font-size: 36px; font-family: Arial;">  ZkSync ETH Escrow</h1>


In [ ]:
#contract-address: 0x32400084C286CF3E17e7B677ea9583e60a000324
#created at 13-02-2023 #at block 16621828


In [22]:
#date of the contract deployment
start_date = '2023-02-13'

# Get the index position of the start date in the DataFrame
start_index = block_numbers_df[block_numbers_df['Date'] == start_date].index[0]

# Slice the DataFrame from the start index onwards
block_numbers_slice = block_numbers_df.iloc[start_index:]

# New function to retrieve Ether balance of a smart contract at a specific block number
def get_contract_eth_balance(escrow_checksum, block_number):
    return web3.eth.get_balance(escrow_checksum, block_identifier=block_number)


escrow = "0x32400084C286CF3E17e7B677ea9583e60a000324"
escrow_checksum = web3.to_checksum_address(escrow)

# Retrieve the token and Ether balances for each token address and block number, and store the results in a DataFrame
token_address = "ether"
token_name = "ethereum"
token_decimals = 18

balances = []
for i, row in block_numbers_slice.iterrows():

    block_number = row['Block Number']
    eth_balance = get_contract_eth_balance(escrow_checksum, block_number) / 10**token_decimals # Convert Wei to Ether
    print(f"Retrieving balances for {row['Date']} - Block Number:{row['Block Number']} - balance:{eth_balance}  ")    
    balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': token_name,
            'token_decimals': token_decimals,
            'token_balance': eth_balance,
        })
    time.sleep(0.5)
        
eth_escrow_balances_per_day = pd.DataFrame(balances)


Retrieving balances for 2023-02-13 - Block Number:16623274 - balance:0.0  
Retrieving balances for 2023-02-14 - Block Number:16630422 - balance:0.0  
Retrieving balances for 2023-02-15 - Block Number:16637574 - balance:0.33  
Retrieving balances for 2023-02-16 - Block Number:16644732 - balance:3.63580942575  
Retrieving balances for 2023-02-17 - Block Number:16651829 - balance:3.91836660125  
Retrieving balances for 2023-02-18 - Block Number:16658938 - balance:3.91836660125  
Retrieving balances for 2023-02-19 - Block Number:16666055 - balance:3.91836660125  
Retrieving balances for 2023-02-20 - Block Number:16673184 - balance:4.41836660125  
Retrieving balances for 2023-02-21 - Block Number:16680299 - balance:4.65358364025  
Retrieving balances for 2023-02-22 - Block Number:16687395 - balance:4.99242081975  
Retrieving balances for 2023-02-23 - Block Number:16694513 - balance:5.035594633524039  
Retrieving balances for 2023-02-24 - Block Number:16701598 - balance:10.535594633524038  


In [24]:
eth_escrow_balances_per_day.to_csv('EthEscrow/eth_escrow_balances_per_day.csv', index=False)

<h1 style="color: blue; font-size: 36px; font-family: Arial;"> Combining the escrows</h1>


In [20]:
eth_escrow_balances_per_day = pd.read_csv('EthEscrow/eth_escrow_balances_per_day.csv')

eth_escrow_balances_per_day = eth_escrow_balances_per_day.rename(columns={'token_balance': 'balance'})
eth_escrow_balances_per_day['escrow_address'] = '0x32400084C286CF3E17e7B677ea9583e60a000324'
eth_escrow_balances_per_day['escrow_address_rdbl'] = 'ZkSyncEthEscrow'

In [53]:
erc20_escrow_balances_per_day = pd.read_csv('Erc20Escrow/erc20_escrow_token_balances_per_day.csv')

erc20_escrow_balances_per_day = erc20_escrow_balances_per_day.rename(columns={'token_balance': 'balance'})
erc20_escrow_balances_per_day['escrow_address'] = '0x57891966931Eb4Bb6FB81430E6cE0A03AAbDe063'
erc20_escrow_balances_per_day['escrow_address_rdbl'] = 'ZkSyncErc20Escrow'
erc20_escrow_balances_per_day = erc20_escrow_balances_per_day.drop(columns=['has_error'])


In [56]:
# create a list of the dataframes
dfs = [erc20_escrow_balances_per_day,
       eth_escrow_balances_per_day
      ]

# concatenate the dataframes along the rows
zksync_escrow_balances = pd.concat(dfs, ignore_index=True)
zksync_escrow_balances = zksync_escrow_balances[zksync_escrow_balances['date'] < '2023-04-30']

In [4]:
zksync_escrow_balances['token_address'] = zksync_escrow_balances['token_address'].apply(lambda x: web3.to_checksum_address(x) if x != 'ether' else x)

price_data = pd.read_csv('../../utils/price-data/price_data.csv') 
price_data['token_address'] = price_data['token_address'].apply(lambda x: web3.to_checksum_address(x) if x != 'ether' else x)


In [5]:
zksync_escrow_balances['date'] = pd.to_datetime(zksync_escrow_balances['date'])
price_data['timestamp'] = pd.to_datetime(price_data['timestamp'])
price_data['token_address'] = price_data['token_address'].astype(str)
zksync_escrow_balances['token_address'] = zksync_escrow_balances['token_address'].astype(str)



In [6]:
zksync_escrow_balances.shape

(1333, 8)

In [12]:
zksync_escrows_final = zksync_escrow_balances.merge(price_data, left_on=['token_address', 'date'], right_on=['token_address', 'timestamp'])

In [65]:
zksync_escrows_final.shape

(1333, 10)

In [13]:
zksync_escrows_final['balance_usd'] = zksync_escrows_final['balance'] * zksync_escrows_final['price']

In [4]:
#can be disregarded and will be overwritten with the correct value in eth in the analysis part

ksync_escrows_final['token_amount'] = zksync_escrows_final['balance'] * 10 ** zksync_escrows_final['token_decimals']

def wei_to_eth(wei_value):
    # 1 Ether (ETH) is equivalent to 10^18 Wei
    eth_value = wei_value * 1e-18
    return eth_value

# Apply the conversion function and add a new column
zksync_escrows_final["balance_eth"] = zksync_escrows_final["token_amount"].apply(wei_to_eth)

### save final df

In [ ]:
#zksync_escrows_final.to_csv('../../data/zksync_escrows_final.csv', index=False)